In [1]:
%%html
<style>
.h1_cell, .just_text {
    box-sizing: border-box;
    padding-top:5px;
    padding-bottom:5px;
    font-family: "Times New Roman", Georgia, Serif;
    font-size: 125%;
    line-height: 22px; /* 5px +12px + 5px */
    text-indent: 25px;
    background-color: #fbfbea;
    padding: 10px;
}

hr { 
    display: block;
    margin-top: 0.5em;
    margin-bottom: 0.5em;
    margin-left: auto;
    margin-right: auto;
    border-style: inset;
    border-width: 2px;
}
</style>

<h1>
<center>
Module 3: Automating tree-building (Part 1)
</center>
</h1>
<div class=h1_cell>

The goal is to eventually build a tool that will generate a decision tree for us. So instead of us trying to guess what nodes and leaves to put in a tree, the tool will do it for us. The tool I have in mind will incrementally build a tree. First, it will find the best root node. Then it will follow each branch and ask what nodes are the best for  each. Eventually it will halt when every path through the tree ends in a leaf (prediction) node.
<p>
We won't take all that on in this module. Instead we will look at a key piece: judging what node should be added to the tree next. To make this judgement, we need a way to measure the goodness of a node/question.

</div>

In [2]:
import pandas as pd
import os

week = 2  # from last module

home_path =  os.path.expanduser('~')

file_path = '/Dropbox/cis399_ds1_f17/notebook_history/'

file_name = 'titanic_wrangled_w'+str(week)+'.csv'

titanic_table = pd.read_csv(home_path + file_path + file_name)

In [3]:
#I am setting the option to see all the columns of our table as we build it.
pd.set_option('display.max_columns', None)
titanic_table.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,no_age,filled_age,emb_C,emb_Q,emb_S,emb_nan,age_bin,age_Child,age_Adult,age_Senior,sex_female,sex_male,pred_1,pred_1_type,ok_child,pclass_1,pclass_2,pclass_3,pclass_nan,pred_2,pred_2_type
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0,22.0,0,0,1,0,Child,1,0,0,0,1,0,true_negative,0,0,0,1,0,0,true_negative
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0,38.0,1,0,0,0,Adult,0,1,0,1,0,1,true_positive,0,1,0,0,0,1,true_positive
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,26.0,0,0,1,0,Child,1,0,0,1,0,1,true_positive,0,0,0,1,0,0,false_negative
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0,35.0,0,0,1,0,Adult,0,1,0,1,0,1,true_positive,0,1,0,0,0,1,true_positive
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,35.0,0,0,1,0,Adult,0,1,0,0,1,0,true_negative,0,0,0,1,0,0,true_negative


<hr>
<h1>
Importing libraries
</h1>
<p>
<div class=h1_cell>
<p>
I asked you to import the week2 library in the last assignment. We need to do the same thing here so we can build on those functions.
</div>

In [4]:
os.chdir(home_path + '/Dropbox/cis399_ds1_f17/week_libraries/datascience_1')
!git pull

Already up-to-date.


In [5]:
import sys
sys.path.append(home_path + '/Dropbox/cis399_ds1_f17/week_libraries/datascience_1')

In [6]:
from week2 import *
%who function

accuracy	 f1	 informedness	 predictor_case	 


<hr>
<h1>
Revisiting the stump
</h1>
<p>
<div class=h1_cell>
Remember the tree we built in module 2? Here it is again.
<p>
<img src="https://www.dropbox.com/s/2940iqadl1nswbq/stump.png?raw=1" width="300" height="300">
<p>
Here is a question I want to ask: Is using sex_female as a root node better than using age_child as a root node? Let's set out to answer that question.

</div>

<div class=just_text>
One thing I will need is the probablity of our target column `Survived`. First I'll build a Series with the counts of the 2 values.
</div>

In [7]:
root_counts = titanic_table['Survived'].value_counts()  # returns a series
root_counts

0    549
1    342
Name: Survived, dtype: int64

In [8]:
def probabilities(counts):
    count_0 = 0 if 0 not in counts else counts[0]  #could have no 0 values
    count_1 = 0 if 1 not in counts else counts[1]
    total = count_0 + count_1
    probs = (0,0) if total == 0 else (1.0*count_0/total, 1.0*count_1/total)  #build 2-tuple
    return probs

In [9]:
root_probs = probabilities(root_counts)
root_probs

(0.6161616161616161, 0.3838383838383838)

<div class=just_text>
All we did is count how many 0s and how many 1s relative to the total number of passengers. When we do the divisions, we get the probabilities we need.
</div>

<hr>
<h1>
Gini to the rescue
</h1>
<p>
<div class=h1_cell>
<p>
There are 2 well-known algorithms for measuring the goodness of a splitter (i.e., a binary column we are contemplating adding to our tree), *`Entropy`* and *`Gini`*. I am going to choose Gini going forward in this course. If you want a good discussion of the difference between the 2, with pointers to more theoretical arguments, this is a good place to start: https://datascience.stackexchange.com/questions/10228/gini-impurity-vs-entropy. The main thing to remember is that you may hear both when on a job interview, so have at least a high-level understanding of Entropy (follow the link).
<p>
The general idea is that we are going to look at the mixture of 0s and 1s in the table before we split with a column. Then we split and look at the mixtures of the 2 sub-tables (one on each branch). Here is the Gini formula (sometimes called *`Gini impurity`*) - P stands for probability (obtained by our `probabilities` function):
<p>
`Gini index = 1 – (P(Target=0)**2 + P(Target=1)**2)`
<p>
We will apply it to the before-split table and the 2 after-split sub-tables. We will end up with 3 separate gini values.
</div>

In [29]:
def gini(counts):
    (p0,p1) = probabilities(counts)
    sum_probs = p0**2 + p1**2
    gini = 1 - sum_probs
    return gini

In [30]:
gini(root_counts)

0.4730129578614428

<div class=just_text>
We now have the Gini score for the Titanic table as a whole. The lower the score, the better.
</div>

<h2>
Do the split
</h2>
<p>
<div class=h1_cell>
We need to build 2 sub-tables that corresond to the 1 (True) and 0 (False) branches of the column we are splitting on. Let's do that now.
</div>

In [12]:
true_table = titanic_table.loc[titanic_table['sex_female'] == 1]  # one branch
false_table = titanic_table.loc[titanic_table['sex_female'] == 0] # the other branch

<div class=just_text>
We now have the 2 sub-tables, one for True branch and one for False branch.
We want to apply gini to each sub-table. First get the counts, then the probabilities. That will give us what we need to get Gini score for each subtable.
</div>

In [13]:
true_counts = true_table['Survived'].value_counts()  # Note using true_table and not titanic_table
true_probs = probabilities(true_counts)
true_probs

(0.25796178343949044, 0.7420382165605095)

In [14]:
false_counts = false_table['Survived'].value_counts()  # using false_table
false_probs = probabilities(false_counts)
false_probs

(0.8110918544194108, 0.18890814558058924)

In [31]:
gini(true_counts)

0.3828350034484158

In [32]:
gini(false_counts)

0.3064437162277842

<div class=just_text>
Cool. We now have 3 separate gini values for the 3 tables involved. Our next step is to combine these values in a way  that gives us an overall goodness-of-the-split score. To do so, we will use something called *`gini information gain`* (GIG). The formula for GIG is as follows:
<p>
`GIG = start_gini − (w_true * gini_true + w_false * gini_false)`
<p>
where w_true is a weight of |true_table|/|titanic_table| (i.e., the size of true_table divided by the size of the starting table) and w_false is a weight of |false_table|/|titanic_table|.
<p>
Unlike plain Gini, which uses 0 as the best score, the gig uses 1 as the best score.
</div>

<h2>
Let's put it all together
</h2>
<p>
<div class=h1_cell>
Let's build a function gig that pulls everything together in one place. The idea is that if you supply a starting table and a column to split on, it will calculate the goodness of the split as the gig score. The bigger, the better.
</div>

In [43]:
def gig(starting_table, split_column, target_column):
    
    #split into two branches, i.e., two sub-tables
    true_table = starting_table.loc[starting_table[split_column] == 1]
    false_table = starting_table.loc[starting_table[split_column] == 0]
    
    #Now see how the target column is divided up in each sub-table (and the starting table)
    true_counts = true_table[target_column].value_counts()  # Note using true_table and not titanic_table
    false_counts = false_table[target_column].value_counts()  # Note using true_table and not titanic_table
    starting_counts = starting_table[target_column].value_counts() 
    
    #compute the gini impurity for the 3 tables
    starting_gini = gini(starting_counts)
    true_gini = gini(true_counts)
    false_gini = gini(false_counts)

    #compute the weights
    starting_size = len(starting_table.index)*1.0
    true_weight = 0.0 if starting_size == 0 else len(true_table.index)/starting_size
    false_weight = 0.0 if starting_size == 0 else len(false_table.index)/starting_size
    
    #wrap it up and put on a bow
    gig = starting_gini - (true_weight * true_gini + false_weight * false_gini)
    
    return gig
    

In [46]:
gig(titanic_table, 'sex_female', 'Survived')

0.1396479574728524

<hr>
<h2>
Compare with another split
</h2>
<p>
<div class=h1_cell>
Ok, this is pretty cool. We can now check out various columns to choose for a split and see what the gig is for each. Let's try ok_child.
</div>

In [45]:
gig(titanic_table, 'ok_child', 'Survived')

0.010321527428076904

<div class=just_text>
It has lower score so "worse" than sex_female: you want low gini scores but high gig scores.
</div>

<hr>
<h2>
Let's go down a branch
</h2>
<p>
<div class=h1_cell>
We were comparing different choices for the root node. Assume we try all columns and sex_female has largest gig score. So we choose it as the root node. We now have 2 branches with a sub-table on each. You guessed it. We can now run the gig on each of the sub-tables to decide what the best choice is. Let's focus on the false branch, i.e., sex_female has value 0. First I'll build the sub-table.
</div>

In [20]:
false_table = titanic_table.loc[titanic_table['sex_female'] == 0]

<div class=just_text>
Now we can start exploring choices for the sub-node
</div>

In [38]:
#try emb_S

gig(false_table, 'emb_S', 'Survived')

0.0013270999683399065

In [22]:
#try pclass_1

gig(false_table, 'pclass_1', 'Survived')

0.01736419615130408

<div class=just_text>
Looks like pclass_1 is the winner between the 2. It has higher gig score then emb_S.
</div>

<hr>
<h1>
Let's do one more thing
</h1>
<p>
<div class=h1_cell>
I think I am fairly close to being able to automate the selection of the best split given the starting table. I'll first decide on the columns I want in the candidate set. Then just map over them to get their gig scores. Finally I'll take the max value.
</div>

In [23]:
titanic_table.columns.values

array(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked', 'no_age', 'filled_age',
       'emb_C', 'emb_Q', 'emb_S', 'emb_nan', 'age_bin', 'age_Child',
       'age_Adult', 'age_Senior', 'sex_female', 'sex_male', 'pred_1',
       'pred_1_type', 'ok_child', 'pclass_1', 'pclass_2', 'pclass_3',
       'pclass_nan', 'pred_2', 'pred_2_type'], dtype=object)

<div class=just_text>
Now I'll just copy and paste from above output and delete the columns that are non-binary or not-useful.
</div>

In [39]:
column_candidates = ['no_age',
       'emb_C', 'emb_Q', 'emb_S', 'emb_nan', 'age_Child',
       'age_Adult', 'age_Senior', 'sex_female', 
        'ok_child', 'pclass_1', 'pclass_2', 'pclass_3',
       'pclass_nan']

<div class=just_text>
I'm ready to do a mapping. This mapping is for the root node.
</div>

In [47]:
gig_scores = map(lambda col: (col, gig(titanic_table, col, 'Survived')), column_candidates)
gig_sorted = sorted(gig_scores, key=lambda item: item[1], reverse=True)
gig_sorted

[('sex_female', 0.1396479574728524),
 ('pclass_3', 0.049137852428292605),
 ('pclass_1', 0.03866453536487213),
 ('emb_C', 0.013388557365673015),
 ('emb_S', 0.011461161069371395),
 ('ok_child', 0.010321527428076904),
 ('pclass_2', 0.004121814088680398),
 ('no_age', 0.0040207042231273915),
 ('emb_nan', 0.0017082345882156735),
 ('age_Child', 0.0006257143143247879),
 ('age_Senior', 0.00048458255066557987),
 ('age_Adult', 0.00019771072603869122),
 ('emb_Q', 6.303036606203349e-06),
 ('pclass_nan', 0.0)]

<div class=just_text>
Now that we know sex_female is the best choice for the root node, we can follow both its true and false branches and rerun the mapping for each. We will take on this problem in the next module.


<h2>
Are we done?
</h2>
<p>
<div class=h1_cell>
Have we chosen the best column for the root? I can say we have chosen the best column from the candidate columns we started with. But we did not have all the columns, e.g., we are missing SibSp, Parch. And there are myriad ways to bin the continuous columns of Fare and Age.
<p>
So, no, I can't say I have the best column. It would not be hard to wrangle the remaining discrete columns to put them in play. It also would not be hard to programmatically generate all possible bins for Fare and Age. However, by my calculation, taking just the integers from 0 to 80 for age, the computational cost is O(80**3).

<hr>
<h1>Write it out</h1>
<div class=h1_cell>

Save the table so can use it in next module.
</div>

In [41]:
import os

week = 3  # change this each week

home_path =  os.path.expanduser('~')

file_path = '/Dropbox/cis399_ds1_f17/notebook_history/'

file_name = 'titanic_wrangled_w'+str(week)+'.csv'

titanic_table.to_csv(home_path + file_path + file_name, index=False)

<hr>
<h1>Next up</h1>
<div class=h1_cell>

    In the next module, we will build on the gig function. We will use it to build a full tree.
</div>